### A description of the problem and a discussion of the background. ###

The restaurant business is very important for the UK hospitality sector. According to Ignite Economics research, restaurants and related sub-sectors generate almost 50% of the UK hospitality sector's GVA (Gross Value Added), while London generates 23.1% of the total UK hospitality GVA (https://www.ukhospitality.org.uk/page/EconomicContributionoftheUKHospitalityIndustry2018). 

But the business is facing strong headwinds, not least those related to the risk of a global recession, uncertainty about Brexit and the spread of coronavirus. According to UHY Hacker Young, the number of restaurant insolvencies has risen by 25% in 2018/19 (https://www.uhy-uk.com/news-events/news/restaurant-insolvencies-jump-25-in-the-last-year-alone). Therefore, it is very important for a starting entrepreneur to pick a right location and relatively quickly establish a permanent client base.

My client is looking to open a restaurant in Finchley, a thriving area in North London where I live. Despite the fact that white people constituted over 60% of the area's population in 2011 (Census data), there does not seem to be many restaurants with European cuisine. My task is to recommend a place for such a restaurant.

### Target audience.

Entrepreneurs looking to open a restaurant in Finchley to capitalise on the lack of restaurants with European cuisine.

### A description of the data and how it will be used to solve the problem. ###

My initial dataframe will probably consist of four columns:

1. Neighbourhood - East Finchley, Finchley Central, West Finchley, North Finchley and Woodside Park.
2. Borough - Barnet (the same for all neighbourhoods).
3. Latitude.
4. Longitude. As the Geocoder package has failed to install multiple times, for each neighbourhood I am planning to use latitude and longitude data from Wiki OpenStreetMap and Wikipedia. 

I will use Foursquare APIs to obtain venues in each neighbourhood and Folium maps for visualisation. I am planning to perform K-Means Clustering for clustering neighbourhoods into three clusters and to make my recommendations based on the availability of European restaurants in each of the clusters.

Import and install libraries

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print("Libraries imported and installed!")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                       

Create a Pandas dataframe

In [3]:
data={'Neighbourhood':['East Finchley','Finchley Central','West Finchley','North Finchley','Woodside Park'],
      'Borough':['Barnet','Barnet','Barnet','Barnet','Barnet'],
      'Latitude':[51.5873,51.6010,51.6092,51.6131,51.6182],
      'Longitude':[-0.1650,-0.1924,-0.1887,-0.1769,-0.1856]}
df=pd.DataFrame(data)
df

,Neighbourhood,Borough,Latitude,Longitude
0,East Finchley,Barnet,51.5873,-0.1650
1,Finchley Central,Barnet,51.6010,-0.1924
2,West Finchley,Barnet,51.6092,-0.1887
3,North Finchley,Barnet,51.6131,-0.1769
4,Woodside Park,Barnet,51.6182,-0.1856


Use Nominatim library to obtain Finchley coordinates

In [4]:
address='Finchley'
geolocator=Nominatim(user_agent="fi_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geographical coordinate of Finchley are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Finchley are 51.5973246, -0.1805587.


Define Foursquare credentials and version

In [5]:
CLIENT_ID=''
CLIENT_SECRET=''
VERSION='20180605'

Create Finchley map with neighbourhoods

In [6]:
map_finchley=folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, location in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label='{}, {}'.format(location, borough)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_finchley)
map_finchley

Create East Finchley dataframe

In [7]:
df.loc[df['Neighbourhood'] == 'East Finchley']

,Neighbourhood,Borough,Latitude,Longitude
0,East Finchley,Barnet,51.5873,-0.165


Get East Finchley's latitude and longitude values

In [8]:
neighbourhood_latitude=df.loc[0,'Latitude']
neighbourhood_longitude=df.loc[0,'Longitude']
neighbourhood_name=df.loc[0,'Neighbourhood']
print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of East Finchley are 51.5873, -0.165.


Get the top 100 venues in East Finchley within a radius of 1000 meters

In [28]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

# displays URL
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZNSY1MVGW3LOTJ53KTL0GYAEVMXSR0TJIZX4HNFIBKYRDSBX&client_secret=NIGSL1FS35O1Y3FKJUDSNCAW5SQKYV5UGOEGUBUN2Y5JRH4R&v=20180605&ll=51.5873,-0.165&radius=1000&limit=100'

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e6b8b0cd03993001b216a33'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'East Finchley',
  'headerFullLocation': 'East Finchley, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 43,
  'suggestedBounds': {'ne': {'lat': 51.59630000900001,
    'lng': -0.15054177619771647},
   'sw': {'lat': 51.57829999099999, 'lng': -0.17945822380228355}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50f002a6e4b0a66769cee32e',
       'name': 'Campbell & Syme',
       'location': {'address': '9 Fortis Green',
        'lat': 51.589616517359204,
        'lng': -0.16320847989856463,
        'labeledLatLngs': [{'label': 'display',
         

Define the function that extracts the venue's category

In [30]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a pandas dataframe

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Campbell & Syme,Coffee Shop,51.589617,-0.163208
1,The Phoenix Cinema,Movie Theater,51.588592,-0.163929
2,Dan & DeCarlo,Café,51.587482,-0.164090
3,The Bald Faced Stag,Pub,51.586177,-0.164845
4,Majjo's,Indian Restaurant,51.589515,-0.163581


Create a function to repeat the same process for all the neighborhoods

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

The code to run the above function on each neighborhood

In [33]:
finchley_venues=getNearbyVenues(names=df['Neighbourhood'],
                               latitudes=df['Latitude'],
                               longitudes=df['Longitude']
                               )

East Finchley
Finchley Central
West Finchley
North Finchley
Woodside Park


The size of the resulting dataframe

In [34]:
print(finchley_venues.shape)
finchley_venues

(219, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,East Finchley,51.5873,-0.1650,Campbell & Syme,51.589617,-0.163208,Coffee Shop
1,East Finchley,51.5873,-0.1650,The Phoenix Cinema,51.588592,-0.163929,Movie Theater
2,East Finchley,51.5873,-0.1650,Dan & DeCarlo,51.587482,-0.164090,Café
3,East Finchley,51.5873,-0.1650,The Bald Faced Stag,51.586177,-0.164845,Pub
4,East Finchley,51.5873,-0.1650,Majjo's,51.589515,-0.163581,Indian Restaurant
5,East Finchley,51.5873,-0.1650,Amici Delicatessen,51.589236,-0.163964,Deli / Bodega
6,East Finchley,51.5873,-0.1650,Indian Rasoi,51.590765,-0.155700,Indian Restaurant
7,East Finchley,51.5873,-0.1650,Bufi Pizzeria,51.589481,-0.163821,Italian Restaurant
8,East Finchley,51.5873,-0.1650,Caffè Nero,51.590296,-0.164227,Coffee Shop
9,East Finchley,51.5873,-0.1650,Table Du Marche,51.592676,-0.166694,French Restaurant


The number of venues returned for each neighborhood

In [35]:
finchley_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
East Finchley,43,43,43,43,43,43
Finchley Central,42,42,42,42,42,42
North Finchley,40,40,40,40,40,40
West Finchley,46,46,46,46,46,46
Woodside Park,48,48,48,48,48,48


The number of unique categories returned from curated venues

In [36]:
print('There are {} uniques categories.'.format(len(finchley_venues['Venue Category'].unique())))

There are 62 uniques categories.


### Analyse each neighbourhood ###

In [37]:
#one hot encoding
finchley_onehot = pd.get_dummies(finchley_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
finchley_onehot['Neighbourhood'] = finchley_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [finchley_onehot.columns[-1]] + list(finchley_onehot.columns[:-1])
finchley_onehot = finchley_onehot[fixed_columns]

finchley_onehot

,Neighbourhood,Auto Garage,Bakery,Bar,Bookstore,Breakfast Spot,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Deli / Bodega,Dessert Shop,Discount Store,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,French Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Movie Theater,Outdoors & Recreation,Park,Persian Restaurant,Pharmacy,Pizza Place,Platform,Portuguese Restaurant,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Soccer Field,Spa,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Turkish Restaurant,Vietnamese Restaurant
0,East Finchley,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,East Finchley,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,East Finchley,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,East Finchley,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,East Finchley,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,East Finchley,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,East Finchley,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,East Finchley,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,East Finchley,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,East Finchley,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [38]:
finchley_grouped = finchley_onehot.groupby('Neighbourhood').mean().reset_index()
finchley_grouped

,Neighbourhood,Auto Garage,Bakery,Bar,Bookstore,Breakfast Spot,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Deli / Bodega,Dessert Shop,Discount Store,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,French Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Movie Theater,Outdoors & Recreation,Park,Persian Restaurant,Pharmacy,Pizza Place,Platform,Portuguese Restaurant,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Soccer Field,Spa,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Turkish Restaurant,Vietnamese Restaurant
0,East Finchley,0.000000,0.023256,0.000000,0.000000,0.000000,0.000000,0.000000,0.046512,0.000000,0.000000,0.069767,0.023256,0.023256,0.000000,0.000000,0.023256,0.000000,0.023256,0.023256,0.023256,0.000000,0.000000,0.023256,0.023256,0.023256,0.00000,0.000000,0.023256,0.000000,0.000000,0.000000,0.046512,0.046512,0.023256,0.000000,0.023256,0.000000,0.023256,0.023256,0.023256,0.069767,0.000000,0.023256,0.023256,0.046512,0.000000,0.139535,0.000000,0.023256,0.023256,0.023256,0.000000,0.000000,0.000000,0.023256,0.000000,0.000000,0.000000,0.000000,0.023256,0.000000,0.000000
1,Finchley Central,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.023810,0.023810,0.023810,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.02381,0.000000,0.000000,0.023810,0.000000,0.023810,0.071429,0.023810,0.047619,0.023810,0.000000,0.023810,0.023810,0.000000,0.000000,0.047619,0.023810,0.023810,0.047619,0.000000,0.000000,0.047619,0.047619,0.023810,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.095238,0.023810
2,North Finchley,0.000000,0.025000,0.025000,0.050000,0.025000,0.000000,0.000000,0.100000,0.025000,0.000000,0.125000,0.000000,0.000000,0.000000,0.025000,0.000000,0.025000,0.000000,0.000000,0.000000,0.025000,0.000000,0.000000,0.075000,0.050000,0.00000,0.000000,0.000000,0.000000,0.025000,0.000000,0.000000,0.025000,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.050000,0.025000,0.000000,0.025000,0.000000,0.000000,0.000000,0.000000,0.050000,0.025000,0.025000,0.025000,0.025000,0.000000,0.000000,0.050000,0.000000
3,West Finchley,0.000000,0.021739,0.000000,0.000000,0.021739,0.021739,0.021739,0.086957,0.043478,0.000000,0.043478,0.000000,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,0.000000,0.021739,0.021739,0.00000,0.021739,0.000000,0.021739,0.000000,0.021739,0.065217,0.043478,0.043478,0.021739,0.000000,0.021739,0.021739,0.000000,0.000000,0.021739,0.021739,0.021739,0.021739,0.000000,0.000000,0.043478,0.021739,0.021739,0.000000,0.000000,0.021739,0.000000,0.065217,0.000000,0.000000,0.000000,0.021739,0.021739,0.000000,0.043478,0.021739
4,Woodside Park,0.020833,0.020833,0.020833,0.041667,0.020833,0.020833,0.020833,0.104167,0.020833,0.020833,0.125000,0.000000,0.000000,0.020833,0.020833,0.000000,0.020833,0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.041667,0.020833,0.00000,0.020833,0.000000,0.000000,0.020833,0.020833,0.000000,0.020833,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.041667,0.000000,0.000000,0.020833,0.020833,0.000000,0.020833,0.000000,0.000000,0.000000,0.020833,0.062500,0.020833,0.020833,0.020833,0.020833,0.000000,0.000000,0.020833,0.000000


Print each neighborhood along with the top 5 most common venues

In [39]:
num_top_venues = 5

for hood in finchley_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = finchley_grouped[finchley_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----East Finchley----
                venue  freq
0                 Pub  0.14
1                Park  0.07
2         Coffee Shop  0.07
3                Café  0.05
4  Italian Restaurant  0.05


----Finchley Central----
                venue  freq
0  Turkish Restaurant  0.10
1   Indian Restaurant  0.07
2         Supermarket  0.07
3         Coffee Shop  0.07
4          Restaurant  0.05


----North Finchley----
                   venue  freq
0            Coffee Shop  0.12
1                   Café  0.10
2          Grocery Store  0.08
3  Portuguese Restaurant  0.05
4              Bookstore  0.05


----West Finchley----
                 venue  freq
0                 Café  0.09
1    Indian Restaurant  0.07
2          Supermarket  0.07
3   Italian Restaurant  0.04
4  Japanese Restaurant  0.04


----Woodside Park----
           venue  freq
0    Coffee Shop  0.12
1           Café  0.10
2    Supermarket  0.06
3  Grocery Store  0.04
4       Pharmacy  0.04




### Put into a pandas dataframe ###

Write a function to sort the venues in descending order

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = finchley_grouped['Neighbourhood']

for ind in np.arange(finchley_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(finchley_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Finchley,Pub,Coffee Shop,Park,Indian Restaurant,Platform,Italian Restaurant,Café,Farmers Market,Health & Beauty Service,Food & Drink Shop
1,Finchley Central,Turkish Restaurant,Indian Restaurant,Coffee Shop,Supermarket,Park,Pizza Place,Japanese Restaurant,Gym / Fitness Center,Pub,Restaurant
2,North Finchley,Coffee Shop,Café,Grocery Store,Pharmacy,Supermarket,Turkish Restaurant,Portuguese Restaurant,Bookstore,Gym / Fitness Center,Sushi Restaurant
3,West Finchley,Café,Indian Restaurant,Supermarket,Japanese Restaurant,Italian Restaurant,Chinese Restaurant,Coffee Shop,Turkish Restaurant,Pub,Vietnamese Restaurant
4,Woodside Park,Coffee Shop,Café,Supermarket,Bookstore,Pharmacy,Grocery Store,Discount Store,Turkish Restaurant,Hotel,Hardware Store


### Cluster Neighborhoods ###

Run k-means to cluster the neighborhood into 3 clusters

In [42]:
# set number of clusters
kclusters = 3

finchley_grouped_clustering = finchley_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(finchley_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 0, 1, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [43]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
finchley_merged = df

# merge finchley_grouped with finchley_data to add latitude/longitude for each neighborhood

finchley_merged = finchley_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

finchley_merged.head()

,Neighbourhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Finchley,Barnet,51.5873,-0.1650,2,Pub,Coffee Shop,Park,Indian Restaurant,Platform,Italian Restaurant,Café,Farmers Market,Health & Beauty Service,Food & Drink Shop
1,Finchley Central,Barnet,51.6010,-0.1924,1,Turkish Restaurant,Indian Restaurant,Coffee Shop,Supermarket,Park,Pizza Place,Japanese Restaurant,Gym / Fitness Center,Pub,Restaurant
2,West Finchley,Barnet,51.6092,-0.1887,1,Café,Indian Restaurant,Supermarket,Japanese Restaurant,Italian Restaurant,Chinese Restaurant,Coffee Shop,Turkish Restaurant,Pub,Vietnamese Restaurant
3,North Finchley,Barnet,51.6131,-0.1769,0,Coffee Shop,Café,Grocery Store,Pharmacy,Supermarket,Turkish Restaurant,Portuguese Restaurant,Bookstore,Gym / Fitness Center,Sushi Restaurant
4,Woodside Park,Barnet,51.6182,-0.1856,0,Coffee Shop,Café,Supermarket,Bookstore,Pharmacy,Grocery Store,Discount Store,Turkish Restaurant,Hotel,Hardware Store


Visualize the resulting clusters

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(finchley_merged['Latitude'], finchley_merged['Longitude'], finchley_merged['Neighbourhood'], finchley_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine clusters ###

Cluster 1

In [45]:
finchley_merged.loc[finchley_merged['Cluster Labels'] == 0, finchley_merged.columns[[0] + list(range(5, finchley_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North Finchley,Coffee Shop,Café,Grocery Store,Pharmacy,Supermarket,Turkish Restaurant,Portuguese Restaurant,Bookstore,Gym / Fitness Center,Sushi Restaurant
4,Woodside Park,Coffee Shop,Café,Supermarket,Bookstore,Pharmacy,Grocery Store,Discount Store,Turkish Restaurant,Hotel,Hardware Store


Cluster 2

In [46]:
finchley_merged.loc[finchley_merged['Cluster Labels'] == 1, finchley_merged.columns[[0] + list(range(5, finchley_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Finchley Central,Turkish Restaurant,Indian Restaurant,Coffee Shop,Supermarket,Park,Pizza Place,Japanese Restaurant,Gym / Fitness Center,Pub,Restaurant
2,West Finchley,Café,Indian Restaurant,Supermarket,Japanese Restaurant,Italian Restaurant,Chinese Restaurant,Coffee Shop,Turkish Restaurant,Pub,Vietnamese Restaurant


Cluster 3

In [47]:
finchley_merged.loc[finchley_merged['Cluster Labels'] == 2, finchley_merged.columns[[0] + list(range(5, finchley_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Finchley,Pub,Coffee Shop,Park,Indian Restaurant,Platform,Italian Restaurant,Café,Farmers Market,Health & Beauty Service,Food & Drink Shop
